# Práctica 2: Manejo básico de ROS2

## Objetivo

Que el alumno haga uso de las funciones básicas de ROS2 (publicador y suscriptor) y de sus aplicaciones con turtlesim y rviz2

### Metas

Que el alumno:
- Integre texto en Markdown dentro de Jupyter
- Integre código de Python dentro de Jupyter
- Haga uso de la terminal de Ubuntu y contruya un script de bash
- Genere un repositorio en GitHub y comparta su archivo de práctica

### Contribución al perfil del egresado

La siguiente práctica contribuye en los siguientes puntos al perfil del egresado:

#### Aptitudes y habilidades

- Para diseñar, construir, operar y mantener los sistemas mecatrónicos y sus componentes.
- Para crear, innovar o evaluar las tecnologías relacionadas con la mecatrónica.

#### Actitudes profesionales

- Ser creativo e innovador.
- Tener confianza en su preparación académica.
- Comprometido con su actualización, superación y competencia profesional.

#### Actitudes de tipo social

- Promover el cambio en la mentalidad frente a la competitividad internacional.

## Rúbrica de evaluación

### La evaluación de la práctica contará de los siguientes puntos

| Elemento | Porcentaje |
| ------:| ----------- |
| **Cuestionario previo** | 15% | 
| **Desarrollo** | 35% |
| **Análisis de resultados**  | 35% |
| **Conclusiones** | 15% |

### Se evaluará con los siguientes criterios:

| Elemento | Malo | Regular | Bueno |
| ------:| ------ | --------| ------|
| **Cuestionario previo** | El trabajo no contiene cuestionario previo o todas las preguntas son incorrectas (0%)| Al menos la mitad de las preguntas son correctas (8%) |  Todas las preguntas son correctas (15%) |
| **Desarrollo** | El trabajo no contiene desarrollo o su planteamiento no concuerda con lo deseado (0%) | El desarrollo está mal planteado o no llega a los resultados esperados (10%) | El desarrollo tiene un planteamiento adecuado y llega a los resultados esperados (35%) |
| **Análisis de resultados**  | El trabajo no contiene análisis de resultados o la información no se está interpretando correctamente (0%) | La interpretación de los resultados es parcial o desorganizada (10%) | Realiza un correcto análisis de los resultados de forma organizada   (35%) |
| **Conclusiones** | El trabajo no contiene conclusiones o no hacen referencia al trabajo desarrollado y los objetivos planteados (0%) | La redacción de las conclusiones es desorganizada o confusa (8%) | Las conclusiones del trabajo son claras y hacen referencia al trabajo desarrollado y los objetivos planteados (15%) | 

## Introducción

### ROS2

ROS 2 (Robot Operating System 2) es un conjunto de bibliotecas y herramientas que permiten desarrollar software para robots de forma modular, distribuida y escalable.
En ROS 2, los programas se comunican entre sí mediante mensajes que viajan sobre una red (incluso entre computadoras distintas), usando un sistema de comunicación llamado DDS (Data Distribution Service).

<img src="imagenes/ROS2_logo.webp" alt = "ROS2" height="100" display= "block"/>

#### Nodos

Un nodo es la unidad básica de ejecución en ROS 2.
Cada nodo es un programa que realiza una tarea específica dentro del sistema del robot: leer un sensor, controlar un motor, procesar imágenes, etc.
En Python, los nodos se definen a partir de la clase base rclpy.node.Node.
~~~python
from rclpy.node import Node

class Nodo(Node):
    def __init__(self):
        super().__init__('nombre_nodo')
        self.get_logger().info("Nodo iniciado")
~~~

#### Tópicos

Los tópicos (topics) son canales de comunicación asíncrona y continua entre nodos.
Un nodo publica mensajes en un tópico, y otros nodos se suscriben para recibirlos.
Todos los mensajes que viajan por un tópico tienen un tipo de dato definido, por ejemplo:
- std_msgs/msg/String
- geometry_msgs/msg/Twist

Por ejemplo:

- Un nodo A (sensor) publica temperaturas en el tópico /temperatura.
- Otro nodo B (monitor) se suscribe a /temperatura y muestra los valores.

Para generar un publicador, se crea un publicador definiendo el tipo de mensaje y el tópico por el que se publica. Después se manda a llamar al publicador para mandar un mensaje.
~~~python
  topic_name = "/topic"
  self.publisher = self.create_publisher(String, topic_name, 10)

  msg = String()
  msg.data = "Message"
  self.publisher.publish(msg)
~~~
Para generar un subscriptor:
~~~python 
  topic_name = "/topic"
  self.subscription = self.create_subscription(
    String, topic_name, self.listener_callback, 10)
~~~

#### Servicios

Los servicios (services) son una forma de comunicación sincrónica (de solicitud–respuesta). Funcionan como una llamada a función entre nodos: uno envía una solicitud (request), y el otro responde con un resultado (response).

Los servicios se usan para operaciones puntuales, no continuas (por ejemplo, mover un robot a una posición, tomar una foto, guardar datos, etc.).

Por ejemplo:

- Un nodo A (cliente) pide: “calcula la suma de 2 y 3”.
- Otro nodo B (servidor) responde: “la suma es 5”.

Para crear un servidor, hay que crear el objeto y la función que se llama al recibir una solicitud

~~~python
  self.srv = self.create_service(AddTwoInts, server_name, self.add_two_ints_callback)
def add_two_ints_callback(self, request:AddTwoInts.Request, response:AddTwoInts.Response):
  response.sum = request.a + request.b
  self.get_logger().info("Recibido: a={}, b={}, respuesta={}".format(request.a, request.b, response.sum))
  return response
~~~

Para crear un cliente, hay que crear el objeto del cliente y la solicitud, enviarla, y definir la función que se llamará al recibir el objeto de la respuesta
~~~python
self.client = self.create_client(AddTwoInts, service_name)
# Esperamos al servicio
while not self.client.wait_for_service(timeout_sec=1.0):
  self.get_logger().info('Esperando al servicio {}...'.format(service_name))
request = AddTwoInts.Request()
request.a = self.a
request.b = self.b
# --- Manda solicitud
future = self.client.call_async(request)
# --- Función que se ejecuta al recibir respuesta
future.add_done_callback(self.callback_result)
~~~

#### Parámetros

Los parámetros son valores configurables que un nodo puede leer o modificar durante su ejecución.
Permiten ajustar el comportamiento del nodo sin cambiar el código.

Por ejemplo:

- Un nodo de control de motores puede tener un parámetro max_velocity de 0.1.
- Otro nodo puede cambiarlo a 0.5 sin interrumpir al primer nodo.

Por ejemplo, si en un nodo queremos que el tópico utilizado sea un parámetro
~~~python
self.declare_parameter("topic_param", "/topic")
topic_name = self.get_parameter("topic_param").value
~~~

Una forma gráfica de ver las comunicaciones en ROS2:

<img src="imagenes/Nodes-TopicandService.gif" height = "400" display = "block" alt = "Vosualización de las comunicaciones en ROS2"/>

### URDF (Unified Robot Description Format):

El URDF es un formato de archivo XML que se utiliza para describir la estructura de un robot, comunmente usado en ROS. Permite definir la composición física de un robot, incluyendo sus eslabones y juntas. Además, permite especificar características como el peso, la geometría, y la disposición espacial de los elementos del robot.

Un archivo URDF básico se construye a partir de **eslabones** y **juntas**.

#### Para definir un eslabón, se utiliza la estructura siguiente:
~~~xml
<!--Definición del eslabón-->
<link name = "nombre_del_eslabón">
  <!--
  Elementos visuales del eslabón 
  (modelos o formas geométricas)
  -->
  <visual>
      <!--
      Posición y rotación del elemento visual
      respecto al origen del eslabón
      -->
      <origin xyz = "x y z" rpy = "gamma beta alfa"/> 
        <!--Elementos visuales asociados a la junta-->
        <geometry> 
        <!--Primitiva visual (caja en este caso)-->
        <box size = "x y z"/> 
     </geometry>
        <!--Material usado para el elemento visual-->
        <material name = "nombre_del_material"> 
          <color rgba = "r g b a"/> 
        </material> 
  </visual> 
</link> 
~~~

Los elementos primitivos son formas geométricas básicas que se utilizan para representar las partes de un robot en URDF. Algunos de los más comunes son:

- **Caja (box)**: Un paralelepípedo definido por su ancho, alto y profundidad.
>~~~xml
>  <box size = "x y z"/> 
>~~~
- **Cilindro (cylinder)**: Definido por su radio y longitud.
>~~~xml
>  <cylinder radius = "r" length="l"/> 
>~~~
- **Esfera (sphere)**: Definida por su radio.
>~~~xml
>  <sphere radius = "r"/> 
>~~~
- **Malla (mesh)**: Permite importar geometrías más complejas en formatos como STL, Collada, obj, entre otros.
>~~~xml
>  <mesh filename = "ruta/al/archivo"/> 
>~~~

Se puede agregar cómo se desplegarán estos elementos a través de un **material**: 
- **Color**: Color plano con el que se despliega el elemento, definido por componentes rgb y transparencia (alfa).
>~~~xml
> <material name = "Nombre"> 
>   <color rgba = "r g b a"/> 
> </material> 
>~~~
- **Textura**: Imagen que se despliega sobre el modelo.
>~~~xml
> <material name = "Nombre"> 
>   <texture filename = "ruta/al/archivo"/> 
> </material> 
>~~~

#### Para una junta: 
~~~xml
<!--Definición de la junta-->
<joint name = "nombre_de_la_junta" type = "tipo_de_junta"> 
  <!--
  Eslabones padre e hijo
  - Padre: Eslabón respecto al cual se mide la rotació o traslación
  - Hijo: Eslabón que se desplaza o rota
  -->
  <parent link = "eslabon_padre_(fijo)"/> 
  <child link = "eslabon_hijo_(movible)"/> 
  <origin xyz = "x y z" rpy = "gamma beta alfa"/> 
  <!--Eje de rotación  (1: se mueve. 0: no se mueve)-->
  <axis xyz = "x y z"/> 
  <!--Límites 
  - effort: par máximo
  - velocity: velocidad absoluta máxima
  - lower-upper: posición mínima y máxima-->
  <limit effort="10.0" lower="-3.14" upper="3.14" velocity="3.14"/>
</joint>
~~~
En el formato URDF, existen varios tipos de juntas que pueden definir el movimiento relativo entre las partes de un robot:

- **Junta fija**: No permite ningún movimiento entre los elementos conectados.
>~~~xml
> <joint name = "nombre_de_la_junta" type = "fixed"> 
>~~~
- **Junta rotacional**: Permite un movimiento de rotación alrededor de un eje.
>~~~xml
> <joint name = "nombre_de_la_junta" type = "revolute"> 
>~~~
- **Junta prismática**: Permite un movimiento lineal a lo largo de un eje.
>~~~xml
> <joint name = "nombre_de_la_junta" type = "prismatic"> 
>~~~
- **Junta continua**: Es una variación de la junta rotacional que permite una rotación sin límites.
>~~~xml
> <joint name = "nombre_de_la_junta" type = "continuous"> 
>~~~

### RVIZ


RViz (ROS Visualization) es una herramienta gráfica dentro del ecosistema de ROS que permite visualizar una amplia gama de información generada por robots. Esta herramienta es esencial para depurar, entender y visualizar los datos que se generan en tiempo real durante la simulación o la operación de un robot en un entorno físico.

<img src="imagenes/rviz.png" alt = "RViz" height="300" display= "block"/>


RViz soporta muchos tipos de visualizaciones, en este caso veremos:

- **Modelos 3D del robot (URDF)**: Puedes cargar y visualizar la descripción del robot usando su archivo URDF (Unified Robot Description Format). Esto te permite ver un modelo 3D del robot y observar cómo se mueve cada parte del robot (es decir, los eslabones y las juntas). También puedes ver cómo se actualiza en tiempo real cuando el robot cambia de posición.

- **Transformaciones (TF)**: ROS utiliza un sistema de coordenadas llamado TF (transformación de marcos) para definir la relación espacial entre diferentes partes del robot y el entorno. En RViz, puedes visualizar estos sistemas de referencia y las transformaciones entre ellos, lo que es útil para entender cómo se mueven y orientan los diferentes componentes del robot.
Esto incluye visualizar el origen de coordenadas del robot y sus partes móviles.

## Cuestionario previo

### Responder de forma breve las siguientes preguntas:

¿Qué es un ejecutor (executor) en ROS2?

RESPUESTA:
En ROS2, un executor es el componente encargado de gestionar la ejecución de los nodos y de procesar los callbacks (suscripciones, timers, servicios, acciones, etc.).
El ejecutor revisa continuamente si algún evento está listo para ejecutarse y lo despacha de forma ordenada. En pocas palabras, es el “orquestador” que decide cuándo y cómo se ejecutan las funciones asociadas a un nodo.
También permite diferentes modelos de ejecución, como un solo hilo (SingleThreadedExecutor) o múltiples hilos (MultiThreadedExecutor).

¿Qué es la ejecución por hilos en Python?

RESPUESTA:
La ejecución por hilos (threading) en Python consiste en dividir un programa en varias líneas de ejecución que pueden correr de forma “paralela” dentro del mismo proceso.
Cada hilo ejecuta su propio flujo de instrucciones, lo que permite que varias tareas avancen de manera concurrente, como escuchar datos, atender eventos o realizar cálculos mientras la aplicación principal sigue funcionando.

¿Qué ventajas tiene ejecutar un programa en un hilo secundario?

RESPUESTA:
Las principales ventajas son:

Permite no bloquear el hilo principal del programa.

Facilita ejecutar tareas que requieren esperar (I/O, sensores, mensajes ROS), sin detener la interfaz o la lógica principal.

Mejora la concurrencia, ya que distintas partes del código pueden avanzar al mismo tiempo.

Es útil para tareas repetitivas o de larga duración sin afectar al resto del sistema.

En ROS2, permite que un nodo procese múltiples callbacks simultáneamente cuando se usa un ejecutor multihilo.

¿Qué pasaría si se utiliza un comando que se queda en periodo de espera (como rclpy.spin()) dentro de un entorno de celdas como Jupyter?

RESPUESTA:
Si ejecutas rclpy.spin() (o cualquier comando que nunca termina) dentro de una celda de Jupyter, la celda quedará bloqueada indefinidamente.
Eso significa que:

No podrás ejecutar ninguna otra celda.

La interfaz de Jupyter se “congela” esperando a que termine el spin (y nunca termina).

Para recuperar el control, debes interrumpir el kernel o reiniciarlo.

Por eso, en Jupyter normalmente se usa threading o MultiThreadedExecutor para que la ejecución continua de ROS2 corra en un hilo aparte y la celda no quede bloqueada.


## Desarrollo

### 1. Tópicos en ROS2

En este primera parte, se usarán las funciones básicas de ROS2 dentro de jupyter

Para ello, en celdas separadas se realizarán las siguientes acciones:

- Inicializar ros2 e importar dependencias (nodos, mensajes, ejecución por hilos y ejecutor de ros2)
~~~python
import rclpy
from rclpy.node import Node
from std_msgs.msg import String
import threading
from rclpy.executors import MultiThreadedExecutor
~~~
- Crear una clase que herede de nodo, que implemente un publicador y sus funciones
~~~python
  class Publisher(Node):
    def __init__(self):
      # Constructor de inicio
    def timer_callback(self):
      # Función de timer para publicar periódicamente cada segundo
~~~
- Crear una clase que herede de nodo, que implemente un subscriptor y sus funciones
~~~python
  class Subscriber(Node):
    def __init__(self):
      # Constructor
    def listener_callback(self, msg):
      # Función que se llama al recibir un mensaje
~~~
- Verificar si ROS2 está inicializado. Si no, inicializarlo. Utilizar la función rclpy.ok(), que nos devuelve el estado de ROS2
~~~python
  if not rclpy.ok():
    rclpy.init(args=None)
~~~
- Instanciar un ejecutor de ROS2 que se encargue de varios nodos de forma simultánea, y ejecutarlo en segundo plano para no bloquear la terminal
~~~ python
  executor = MultiThreadedExecutor()
  thread = threading.Thread(target=executor.spin, daemon=True)
  thread.start()
~~~
- Instanciar los nodos publicador y suscriptor y agregarlos al ejecutor
~~~python
  pub_node = SimplePublisher()
  sub_node = SimpleSubscriber()
  executor.add_node(sub_node)
  executor.add_node(pub_node)
~~~
- Destruir los nodos para que dejen de ejecutarse
~~~python
  pub_node.destroy_node()
  sub_node.destroy_node()
~~~
- Que detenga el proceso de ROS2 en caso de que siga activo (para evitar procesos en segundo plano)
~~~python
  if rclpy.ok():
    rclpy.shutdown()
~~~

Como probatorio: 
- Las celdas con código en esta sección del archivo
- Dejar los mensajes de salida que se imprimirán al agregar los nodos al ejecutor

In [1]:
import rclpy
from rclpy.node import Node
from std_msgs.msg import String
import threading
from rclpy.executors import MultiThreadedExecutor

In [2]:
class Publisher(Node):
  def __init__(self):
    super().__init__('simple_publisher')
    topic_name = "/topic"
    self.publisher = self.create_publisher(String, topic_name, 10)
    self.timer = self.create_timer(1.0, self.timer_callback)
    self.get_logger().info('Publicador iniciado en {}'.format(topic_name))
    self.count = 0
    
  def timer_callback(self): 
    msg = String()
    msg.data = "Mensaje No. {}".format(self.count)
    self.publisher.publish(msg)
    self.get_logger().info('Publicado: {}'.format(msg.data))
    self.count += 1
      
class Subscriber(Node):
  def __init__(self):
    super().__init__('simple_subscriber')
    topic_name = "/topic"
    self.subscription = self.create_subscription(String, topic_name, self.listener_callback, 10)
    self.get_logger().info('Suscriptor escuchando topico: {}'.format(topic_name))
    
  def listener_callback(self, msg):
    self.get_logger().info(f'Recibido: "{msg.data}"')

In [3]:
if not rclpy.ok():
  rclpy.init(args=None)

In [4]:
executor = MultiThreadedExecutor()
thread = threading.Thread(target=executor.spin, daemon=True)
thread.start()

In [5]:
pub_node = Publisher()
sub_node = Subscriber()
executor.add_node(sub_node)
executor.add_node(pub_node)

[INFO] [1763054433.658634892] [simple_publisher]: Publicador iniciado en /topic
[INFO] [1763054433.662336979] [simple_subscriber]: Suscriptor escuchando topico: /topic


True

[INFO] [1763054434.648661172] [simple_publisher]: Publicado: Mensaje No. 0
[INFO] [1763054434.653462903] [simple_subscriber]: Recibido: "Mensaje No. 0"
[INFO] [1763054435.642990260] [simple_subscriber]: Recibido: "Mensaje No. 1"
[INFO] [1763054435.646511903] [simple_publisher]: Publicado: Mensaje No. 1
[INFO] [1763054436.648892761] [simple_publisher]: Publicado: Mensaje No. 2
[INFO] [1763054436.654141202] [simple_subscriber]: Recibido: "Mensaje No. 2"
[INFO] [1763054437.644193495] [simple_subscriber]: Recibido: "Mensaje No. 3"
[INFO] [1763054437.648619150] [simple_publisher]: Publicado: Mensaje No. 3
[INFO] [1763054438.647379529] [simple_publisher]: Publicado: Mensaje No. 4
[INFO] [1763054438.654253640] [simple_subscriber]: Recibido: "Mensaje No. 4"
[INFO] [1763054439.643429494] [simple_publisher]: Publicado: Mensaje No. 5
[INFO] [1763054439.646630693] [simple_subscriber]: Recibido: "Mensaje No. 5"
[INFO] [1763054440.646967760] [simple_subscriber]: Recibido: "Mensaje No. 6"
[INFO] [176

In [6]:
pub_node.destroy_node()
sub_node.destroy_node()

if rclpy.ok():
  rclpy.shutdown()

### 2. Servicios en ROS2

En este segunda parte, se usarán los servicios de ROS2 dentro de jupyter

Para ello, en celdas separadas se realizarán las siguientes acciones:

- Inicializar ros2 e importar dependencias (nodos, mensajes, ejecución por hilos y ejecutor de ros2)
~~~python
import rclpy
from rclpy.node import Node
from std_msgs.msg import String
import threading
from rclpy.executors import MultiThreadedExecutor
~~~
- Crear una clase que herede de nodo, que implemente un servidor y sus funciones
~~~python
  class Server(Node):
    def __init__(self):
      # Constructor
    def listener_callback(self, msg):
      # Función que se llama al recibir un mensaje
~~~
- Crear una clase que herede de nodo, que implemente un cliente y sus funciones
~~~python
  class Client(Node):
    def __init__(self):
      # Constructor de inicio
    def timer_callback(self):
      # Función de timer para enviar una solicitud periódicamente cada segundo
    def callback_result(self, future:Future):
      # Función que se llama al recibir una respuesta a la solicitud enviada
~~~
- Verificar si ROS2 está inicializado. Si no, inicializarlo. Utilizar la función rclpy.ok(), que nos devuelve el estado de ROS2
~~~python
  if not rclpy.ok():
    rclpy.init(args=None)
~~~
- Instanciar un ejecutor de ROS2 que se encargue de varios nodos de forma simultánea, y ejecutarlo en segundo plano para no bloquear la terminal
~~~ python
  executor = MultiThreadedExecutor()
  thread = threading.Thread(target=executor.spin, daemon=True)
  thread.start()
~~~
- Instanciar los nodos servidor y cliente, y agregarlos al ejecutor
~~~python
  server_node = Server()
  client_node = Client()
  executor.add_node(server_node)
  executor.add_node(client_node)
~~~
- Destruir los nodos para que dejen de ejecutarse
~~~python
  server_node.destroy_node()
  client_node.destroy_node()
~~~
- Que detenga el proceso de ROS2 en caso de que siga activo (para evitar procesos en segundo plano)
~~~python
  if rclpy.ok():
    rclpy.shutdown()
~~~

Como probatorio: 
- Las celdas con código en esta sección del archivo
- Dejar los mensajes de salida que se imprimirán al agregar los nodos al ejecutor

In [1]:
import rclpy
from rclpy.node import Node
from rclpy.task import Future
from std_msgs.msg import String
import threading
from rclpy.executors import MultiThreadedExecutor
from example_interfaces.srv import AddTwoInts

In [2]:
class Client(Node):
    def __init__(self):
        super().__init__('simple_service_client')
        service_name = "/sum_two_ints"
        self.a = 10
        self.b = 15
        self.count = 0
        self.client = self.create_client(AddTwoInts, service_name)
        while not self.client.wait_for_service(timeout_sec=1.0):
            self.get_logger().info(f"Esperando al servicio {service_name}...")

        self.get_logger().info(f"Cliente conectado a {service_name}")
        self.timer = self.create_timer(1.0, self.timer_callback)
                 
    def timer_callback(self):
        self.count += 1
        req = AddTwoInts.Request()
        req.a = self.a
        req.b = self.b + self.count
        future = self.client.call_async(req)
        future.add_done_callback(self.callback_result)
        
    def callback_result(self, future):
      try:
        response = future.result()
        self.get_logger().info(
            f"Resultado recibido: {self.a} + {self.b + self.count} = {response.sum}"
        )
      except Exception as e:
        self.get_logger().error(f"Error al recibir respuesta: {e}")

class Server(Node):
    def __init__(self):
        super().__init__('simple_service_server')
        service_name = "/sum_two_ints"
        self.srv = self.create_service(AddTwoInts, service_name, self.listener_callback)
        self.get_logger().info("El servicio está disponible en: {}".format(service_name))  
        
    def listener_callback(self, request, response):
        response.sum = request.a + request.b
        self.get_logger().info(f"Recibido: a={request.a}, b={request.b}, sum={response.sum}")
        return response

In [3]:
if not rclpy.ok():
    rclpy.init(args=None)

In [4]:
executor = MultiThreadedExecutor()
thread = threading.Thread(target=executor.spin, daemon=True)
thread.start()

In [5]:
server_node = Server()
client_node = Client()
executor.add_node(server_node)
executor.add_node(client_node)

[INFO] [1763084694.950204392] [simple_service_server]: El servicio está disponible en: /sum_two_ints
[INFO] [1763084694.956845624] [simple_service_client]: Cliente conectado a /sum_two_ints


True

[INFO] [1763084695.961100072] [simple_service_server]: Recibido: a=10, b=16, sum=26
[INFO] [1763084695.965288941] [simple_service_client]: Resultado recibido: 10 + 16 = 26
[INFO] [1763084696.963086002] [simple_service_server]: Recibido: a=10, b=17, sum=27
[INFO] [1763084696.967780753] [simple_service_client]: Resultado recibido: 10 + 17 = 27
[INFO] [1763084697.965332791] [simple_service_server]: Recibido: a=10, b=18, sum=28
[INFO] [1763084697.972735658] [simple_service_client]: Resultado recibido: 10 + 18 = 28
[INFO] [1763084698.963162069] [simple_service_server]: Recibido: a=10, b=19, sum=29
[INFO] [1763084698.970594031] [simple_service_client]: Resultado recibido: 10 + 19 = 29
[INFO] [1763084699.967419372] [simple_service_server]: Recibido: a=10, b=20, sum=30
[INFO] [1763084699.973473199] [simple_service_client]: Resultado recibido: 10 + 20 = 30
[INFO] [1763084700.967062463] [simple_service_server]: Recibido: a=10, b=21, sum=31
[INFO] [1763084700.972486610] [simple_service_client]: R

In [6]:
server_node.destroy_node()
client_node.destroy_node()

if rclpy.ok():
    rclpy.shutdown()

### Códigos utilizados


### 2. URDF y RViz

En esta sección, se creará un archivo URDF de un robot tipo RRR con la siguiente disposición:
<div align="center">
<img src="imagenes/modelo_robot.png" alt = "Robot RRR" width="300" height="300"/>
</div>

** Se pueden utilizar formas geométricas sencillas
** Mantener las dimensiones menores a 0.5 \[m]. 

#### Despliegue de un modelo URDF con configuraciones predeterminadas en RViz
Ahora, se tomará el archivo URDF realizado y se desplegará en RViz utilizando algunas configuraciones predeterminadas. 
Para desplegar el archivo en RViz, se deben instalar algunas librerías:

Para descargar un paquete que contiene un launch para desplegar al robot
> $ sudo apt-get install ros-humble-urdf-tutorial

Instalar la librería que permite a ROS manipular los URDF (xacro)
> $ sudo apt install ros-humble-xacro

Hay que verificar las juntas del archivo. RViz requiere que las juntas tengan límites definidos, agregando el parámetro
``` xml 
<limit effort="XX" velocity="XX" lower="XX" upper="XX" />
```
Siendo "XX" los valores deseados para cada parámetro.

Con estos cambios, se puede correr el despliegue del modelo con un archivo *.launch del paquete urdf-tutorial 

> $ ros2 launch urdf_tutorial display.launch.py model:=/home/robousr/<ruta_del_modelo>/<nombre_del_modelo>.urdf



Como entregable, agregar una imagen del modelo desplegado en RViz y una celda con el código xml del modelo urdf

<div align="center">
<img src="imagenes/modelo_desplegado.png" alt = "Primer despliegue en RViz" width="300" height="300" display= "block"/>
</div>

### Codigos utilizados

~~~xml
<robot name="Practica_2_robot"> 
  <!--Definición de eslabones--> 
<link name = "base_link">
  <visual>
      <origin xyz = "0 0 0.05" rpy = "0 0 0"/> 
        <geometry> 
          <box size = "0.2 0.2 0.1"/> 
        </geometry> 
        <material name = "Black"> 
        <color rgba = "0 0 0 1.0"/> 
      </material> 
    </visual> 
</link> 
 
<link name = "shoulder_link"> 
   <visual> 
      <origin xyz = "0 0 0.15" rpy = "0 0 0"/> 
    <geometry> 
      <box size = "0.03 0.03 0.3"/> 
     </geometry> 
      <material name = "Cyan"> 
        <color rgba = "0 1 1 1.0"/> 
      </material> 
    </visual> 
  </link> 

<link name = "arm_link"> 
   <visual> 
      <origin xyz = "0.15 0 0" rpy = "0 0 0"/> 
    <geometry> 
      <box size = "0.3 0.03 0.03"/> 
     </geometry> 
      <material name = "Magenta"> 
        <color rgba = "1 0 1 1.0"/> 
      </material> 
    </visual> 
  </link> 
<link name = "forearm_link"> 
   <visual> 
      <origin xyz = "0.15 0 0" rpy = "0 0 0"/> 
    <geometry> 
      <box size = "0.3 0.03 0.03"/> 
     </geometry> 
      <material name = "Yellow"> 
        <color rgba = "1 1 0 1.0"/> 
      </material> 
    </visual> 
  </link> 
 
 <link name = "hand_link"> 
   <visual> 
      <origin xyz = "0 0 0" rpy = "0 0 0"/> 
    <geometry> 
      <box size = "0.03 0.03 0.3"/> 
     </geometry> 
      <material name = "White"> 
        <color rgba = "1 1 1 1.0"/> 
      </material> 
    </visual> 
  </link> 
 
  <!--Definició de juntas--> 

<joint name = "shoulder_joint" type = "revolute"> 
  <parent link = "base_link"/> 
  <child link = "shoulder_link"/> 
  <origin xyz = "0 0 0.1" rpy = "0 0 0"/> 
  <axis xyz = "0 0 1"/> 
  <limit effort="10.0" lower="-3.14" upper="3.14" velocity="3.14"/>
</joint>

<joint name = "arm_joint" type = "fixed"> 
  <parent link = "shoulder_link"/> 
  <child link = "arm_link"/> 
  <origin xyz = "0 0 0.3" rpy = "0 0 0"/> 
  <axis xyz = "0 0 1"/> 
  <limit effort="10.0" lower="-3.14" upper="3.14" velocity="3.14"/>
</joint>
<joint name = "forearm_joint" type = "revolute"> 
  <parent link = "arm_link"/> 
  <child link = "forearm_link"/> 
  <origin xyz = "0.28 0 0.03" rpy = "0 0 0"/> 
  <axis xyz = "0 0 1"/> 
  <limit effort="10.0" lower="-3.14" upper="3.14" velocity="3.14"/>
</joint>
<joint name = "hand_joint" type = "prismatic"> 
  <parent link = "forearm_link"/> 
  <child link = "hand_link"/> 
  <origin xyz = "0.3 0 0" rpy = "0 0 0"/> 
  <axis xyz = "0 0 1"/> 
  <limit effort="10.0" lower="-0.15" upper="0.15" velocity="3.14"/>
</joint>

</robot>
~~~

<div align="center">
<img src="imagenes/img1.png" alt = "Primer despliegue en RViz" width="500" height="500" display= "block"/>
</div>

<div align="center">
<img src="imagenes/img2.png" alt = "Primer despliegue en RViz" width="500" height="500" display= "block"/>
</div>

<div align="center">
<img src="imagenes/img3.png" alt = "Primer despliegue en RViz" width="500" height="500" display= "block"/>
</div>

## Análisis de resultados

¿Cuáles son las ventajas o desventajas de usar el protocolo de comunicación publicador/suscriptor o cliente/servicio?

RESPUESTA:
El protocolo publicador/suscriptor ofrece la ventaja de permitir una comunicación desacoplada, continua y asíncrona. Esto es útil cuando varios nodos necesitan recibir información al mismo tiempo o cuando los datos se generan de manera periódica (por ejemplo, sensores, odometría o estados del robot). Además, facilita la escalabilidad porque cualquier nodo puede suscribirse sin modificar el publicador.
Como desventaja, este modelo no garantiza una respuesta inmediata ni confirmación, ya que los mensajes simplemente se transmiten al tópico sin asegurar que un suscriptor los procese.

Por otro lado, el protocolo cliente/servicio es ideal cuando se necesita una acción puntual, donde un nodo solicita algo y otro responde (similar a una función remota). Su ventaja es que ofrece sincronización y confirmación, ya que el cliente recibe una respuesta directa.
Sin embargo, como desventaja, no es adecuado para información continua y puede bloquear la ejecución si la respuesta tarda demasiado. Es un modelo menos flexible para datos que cambian en tiempo real y no escala tan bien para múltiples nodos simultáneos.

¿Cuál es la convención de ángulos que utilizan los archivos URDF para los ángulos de las juntas respecto al eslabón padre? (Intrínsecos/extrínsecos y el orden)

RESPUESTA:
Los archivos URDF utilizan rotaciones extrínsecas en el orden roll–pitch–yaw (RPY) para describir la orientación de un enlace o junta respecto a su eslabón padre.
Esto significa que las rotaciones se aplican respecto a los ejes fijos del sistema de coordenadas del eslabón padre y en el siguiente orden:

Roll (rotación alrededor del eje X del padre)

Pitch (rotación alrededor del eje Y del padre)

Yaw (rotación alrededor del eje Z del padre)

Este sistema facilita interpretar la cadena cinemática y mantiene compatibilidad con la mayoría de las herramientas de simulación y visualización que emplea ROS, como RViz y Gazebo.

¿Qué utilidad tiene describir un robot en un archivo URDF?

RESPUESTA:
Describir un robot en un archivo URDF permite representar su estructura física, sus enlaces, juntas, masas, colisiones y geometrías de una forma estándar.
Esto es útil porque permite:

Visualizar el robot en herramientas como RViz o Gazebo.

Generar modelos cinemáticos y dinámicos de manera automática.

Integrar sensores, actuadores y propiedades físicas sin escribir código adicional.

Asegurar que todos los nodos de ROS utilicen la misma descripción del robot, evitando inconsistencias.

Facilitar simulaciones, planeación de movimiento y pruebas sin necesidad del robot físico.

En resumen, el URDF es la “descripción oficial” del robot dentro del ecosistema ROS y es fundamental para cualquier tarea de simulación, control o percepción.




## Conclusiones

El objetivo de esta práctica consistió en comprender y aplicar los mecanismos fundamentales de comunicación en ROS2, analizar el funcionamiento interno de los ejecutores, explorar la ejecución concurrente mediante hilos y reconocer la importancia de describir correctamente un robot utilizando un archivo URDF. A partir del desarrollo de las actividades, se logró cumplir completamente este propósito, ya que se integraron conocimientos teóricos con ejercicios prácticos que permitieron observar de manera directa cómo ROS2 gestiona la información, coordina procesos y estructura el modelo físico de un robot.

El trabajo con los protocolos publicador/suscriptor y cliente/servicio permitió identificar las diferencias operativas entre ambos enfoques. El modelo publicador/suscriptor evidenció su eficacia para transmitir datos continuamente sin bloquear la ejecución, lo cual es indispensable en sistemas robóticos que requieren flujos constantes de información, como sensores o telemetría. En contraste, el mecanismo cliente/servicio demostró ser adecuado para solicitudes puntuales, donde se necesita sincronización y una respuesta explícita. Comprender estas diferencias fue esencial para alcanzar el objetivo de seleccionar de manera consciente el protocolo adecuado según el tipo de tarea robótica que se quiera implementar.

Asimismo, el análisis del funcionamiento de los ejecutores en ROS2 cumplió una parte clave del objetivo, al mostrar cómo el sistema administra internamente los callbacks y permite la concurrencia mediante modelos de un solo hilo y multihilo. Relacionar este comportamiento con el módulo threading de Python permitió entender por qué ciertos comandos, como rclpy.spin(), pueden bloquear un entorno interactivo como Jupyter, y cómo la ejecución en hilos secundarios evita estos problemas. Esto reforzó la comprensión de la arquitectura interna de ROS2 y la importancia de la ejecución concurrente en sistemas distribuidos.

Finalmente, la construcción y el análisis de un modelo URDF permitieron alcanzar el objetivo relacionado con la descripción física del robot. Se comprendió por qué el URDF es el punto de partida para la visualización, simulación, planeación de movimiento y publicación del estado del robot. El uso de rotaciones extrínsecas en el orden roll–pitch–yaw también ayudó a interpretar con mayor claridad la estructura cinemática del modelo, fortaleciendo la capacidad para modificar, corregir o ampliar la descripción física de un robot real o simulado.

En conjunto, la práctica no solo permitió cumplir el objetivo planteado, sino que también brindó una visión integrada de cómo la comunicación, la concurrencia y la descripción estructural del robot se relacionan entre sí dentro del ecosistema ROS2. Este aprendizaje constituye una base sólida para el desarrollo de aplicaciones robóticas más complejas, donde la correcta selección de protocolos, el uso eficiente de ejecutores y la descripción precisa del robot son elementos esenciales para un funcionamiento robusto y profesional.



## Bibliografía 

[1] Open Robotics, “Executors — ROS 2 Documentation: Humble,” 2024. [Online]. Disponible: https://docs.ros.org/en/humble/Concepts/Intermediate/About-Executors.html

[2] Open Robotics, “Bibliotecas cliente — Descripción general de rclpy,” 2024. [En línea]. Disponible en: https://docs.ros.org/en/humble/Concepts/Basic/About-Client-Libraries.html

[3] Open Robotics, “Comunicación en ROS 2 — Tópicos, servicios y acciones,” 2024. [En línea]. Disponible en: https://docs.ros.org/en/humble/Concepts/Basic/About-Communication.html

[4] Open Robotics, “URDF — Formato Unificado de Descripción de Robots,” 2024. [En línea]. Disponible en: https://docs.ros.org/en/humble/How-To-Guides/URDF.html

[5] Open Robotics, “Referencia XML de URDF,” 2024. [En línea]. Disponible en: https://wiki.ros.org/urdf/XML